In [ ]:
%env ANYWIDGET_HMR=1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/clicker/clicker_simple.ipynb)

# Clicker — Quick Demo

Pick atomic column positions on a HAADF-STEM image with a hexagonal lattice.
Click on bright atom columns to select up to 5 positions.

In [ ]:
import numpy as np
from quantem.widget import Clicker


def make_haadf_stem(size=256, spacing=18, sigma=2.8):
    """HAADF-STEM image with atomic columns on a hexagonal lattice."""
    y, x = np.mgrid[:size, :size]
    img = np.random.normal(0.08, 0.015, (size, size))
    a1 = np.array([spacing, 0.0])
    a2 = np.array([spacing * 0.5, spacing * np.sqrt(3) / 2])
    for i in range(-1, size // spacing + 2):
        for j in range(-1, size // spacing + 2):
            cx = i * a1[0] + j * a2[0]
            cy = i * a1[1] + j * a2[1]
            if -spacing < cx < size + spacing and -spacing < cy < size + spacing:
                # Slight intensity variation (like mixed Z columns)
                intensity = 0.7 + 0.3 * ((i + j) % 3 == 0)
                img += intensity * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    # Add scan noise (horizontal streaks)
    scan_noise = np.random.normal(0, 0.01, (size, 1)) * np.ones((1, size))
    img += scan_noise
    return np.clip(img, 0, None).astype(np.float32)


haadf = make_haadf_stem()
w = Clicker(haadf, scale=1.0, max_points=5)
w

## Retrieve selected points

After clicking on atom columns above, run the cell below to get pixel coordinates.
With 3 points you can define a lattice basis: pick an origin, then two neighbors
to get lattice vectors **u** and **v**.

In [ ]:
points = w.selected_points
print(f"Selected {len(points)} point(s):")
for i, p in enumerate(points):
    print(f"  P{i}: x={p['x']:.1f}, y={p['y']:.1f}")

if len(points) >= 3:
    origin = np.array([points[0]["x"], points[0]["y"]])
    u = np.array([points[1]["x"], points[1]["y"]]) - origin
    v = np.array([points[2]["x"], points[2]["y"]]) - origin
    print(f"\nLattice basis (from first 3 points):")
    print(f"  origin = ({origin[0]:.1f}, {origin[1]:.1f})")
    print(f"  u = ({u[0]:.1f}, {u[1]:.1f})  |u| = {np.linalg.norm(u):.1f} px")
    print(f"  v = ({v[0]:.1f}, {v[1]:.1f})  |v| = {np.linalg.norm(v):.1f} px")

## Gallery Mode

Pass multiple images to pick points on each independently.
Click an image to select it, then click to place points.

In [ ]:
# Create 3 HAADF images with different lattice spacings
imgs = [
    make_haadf_stem(size=128, spacing=14),
    make_haadf_stem(size=128, spacing=18),
    make_haadf_stem(size=128, spacing=22),
]
w_gallery = Clicker(imgs, ncols=3, max_points=3, labels=["d=14px", "d=18px", "d=22px"])
w_gallery

In [ ]:
# Retrieve per-image points
for i, pts in enumerate(w_gallery.selected_points):
    print(f"Image {i} ({w_gallery.labels[i]}): {len(pts)} point(s)")
    for j, p in enumerate(pts):
        print(f"  P{j}: x={p['x']:.1f}, y={p['y']:.1f}")